In [252]:
import pandas as pd
import plotly.express as px
import re
import datetime
import numpy as np

from Project.Database import Db

# Example data for the TPM

In [253]:
# pd.read_csv("/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/TPM/Data/ISID.csv")

In [254]:
# pd.read_json(
#     path_or_buf="/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/TPM/output/Experiment_minsup0.1_minconf_0.1/level1.json",
#     orient='name_node')

In [255]:
year1_minutes, meta = Db.load_data(hourly=False, meta=True, year=1)

In [256]:
meta

,Subsystem,Measurement_Location,Parameter,Description,Units,Aggregation_Method,min_value,max_value,Measurement_Floor,Load_Match,Status_Match,Elec_Match
Unnamed: 0,,,,,,,,,,,,
DHW_ClothesWasherColdFlow,DHW,Utility,Flow_Water,The cumulative volume of cold water flowing in...,Gallons,Hour value,0.0000,33.0646,1stFloor,NaN,NaN,NaN
DHW_ClothesWasherHotFlow,DHW,Utility,Flow_Water,The cumulative volume of hot water flowing int...,Gallons,Hour value,0.0000,201.6969,1stFloor,NaN,NaN,NaN
DHW_DishwasherHotFlow,DHW,Kitchen,Flow_Water,The cumulative volume of hot water flowing int...,Gallons,Hour value,0.0000,4.6367,1stFloor,NaN,NaN,NaN
DHW_HeatPumpWaterHeaterEnergyTotal,DHW,Basement,Energy_Electrical,Cumulative energy consumption by heat pump wat...,Wh,Hour value,0.0000,188169.7774,Basement,NaN,NaN,NaN
DHW_HeatPumpWaterHeaterPowerTotal,DHW,Basement,Power_Electrical,Instantaneous power consumption by heat pump w...,W,Average,0.0000,4368.8008,Basement,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
Vent_HRVTempSupplyOut,Ventilation,Basement,Temp_Air,Instantaneous dry bulb temperature in the supp...,degreesC,Average,-0.4606,34.1330,Basement,NaN,NaN,NaN
HVAC_HeatPumpIndoorUnitPower,HVAC,Basement,Power_Electrical,Average power consumption of heat pump indoor ...,W,Average,0.0000,21843.4750,Basement,NaN,NaN,NaN
HVAC_HeatPumpOutdoorUnitPower,HVAC,Outdoor,Power_Electrical,Average power consumption of heat pump outdoor...,W,Average,-68.7461,8624.9120,Outdoor,NaN,NaN,NaN


In [257]:
appliance_condition = (lambda self: (
        (self["Parameter"] == "Status_OnOff") &
        (self["Subsystem"] == "Loads") &
        # (~self.index.str.contains("Child")) &
        # (~self.index.str.contains("Prnt")) &
        (~self.index.str.contains("StatusLatentload"))  #&
    # (~self.index.str.contains("Load_StatusClothesWasher")) &
    # (~self.index.str.contains("Load_StatusDryerPowerTotal"))
))

appliances_statusOnOff = meta.loc[appliance_condition].index.tolist()

In [258]:
minute_appliances_status = year1_minutes[["Timestamp"] + appliances_statusOnOff].copy()

In [259]:
for col in meta["Measurement_Location"].unique():
    appliance_condition = (lambda self: (
            (self["Parameter"] == "Status_OnOff") &
            (self.index.str.contains("Light")) &
            (self["Measurement_Location"] == col)
    ))
    light_cols = meta.loc[appliance_condition].index
    if len(light_cols) > 1:
        minute_appliances_status[col + "Lights"] = minute_appliances_status[light_cols].max(1)
        minute_appliances_status.drop(light_cols, axis=1, inplace=True)
        for attribute in light_cols:
            appliances_statusOnOff.remove(attribute)
        appliances_statusOnOff.append(col + "Lights")

In [260]:
#Extracts day of datetime.
minute_appliances_status["DayOfYear"] = minute_appliances_status.Timestamp.dt.dayofyear

In [261]:
minute_appliances_status["DayOfYear"] = minute_appliances_status.Timestamp.dt.dayofyear
minute_appliances_status["DayOfYear"] = minute_appliances_status["DayOfYear"] - minute_appliances_status["DayOfYear"][0]
minute_appliances_status["DayOfYear"] = np.where(minute_appliances_status["DayOfYear"] < 0,
                                                 minute_appliances_status["DayOfYear"] + 365,
                                                 minute_appliances_status["DayOfYear"])
minute_appliances_status.loc[500000:].loc[lambda self: self["DayOfYear"] == 0] = 365
minute_appliances_status

/var/folders/bd/t85lw5j14_vc55x5tn1xbmr00000gn/T/ipykernel_1126/3932440917.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minute_appliances_status.loc[500000:].loc[lambda self: self["DayOfYear"] == 0] = 365


,Timestamp,Load_StatusApplianceCooktop,Load_StatusApplianceDishwasher,Load_StatusApplianceOven,Load_StatusApplianceRangeHood,Load_StatusBA1Lights,Load_StatusBA2Lights,Load_StatusBR2Lights,Load_StatusBR3Lights,Load_StatusBR4Lights,...,Load_StatusSensHeatPrntADOWN,Load_StatusSensHeatPrntAUP,Load_StatusSensHeatPrntBDOWN,Load_StatusSensHeatPrntBUP,Load_StatusClothesWasher,Load_StatusDryerPowerTotal,KitchenLights,LivingRoomLights,MasterBedroomLights,DayOfYear
0,2013-07-01 00:01:14+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
1,2013-07-01 00:02:14+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
2,2013-07-01 00:03:14+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
3,2013-07-01 00:04:14+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
4,2013-07-01 00:05:14+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518787,2014-06-30 23:29:22+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0,0,0.0,0.0,0.0,364
518788,2014-06-30 23:30:22+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0,0,0.0,0.0,0.0,364
518789,2014-06-30 23:31:22+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0,0,0.0,0.0,0.0,364
518790,2014-06-30 23:59:22+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,364


In [262]:
# minute_appliances_status['Timestamp'] = (pd.to_datetime(minute_appliances_status["Timestamp"],
#                                                        format="%Y-%m-%d %H:%M:%S%z", utc=True).astype("int") / 1e9).astype("int")

In [263]:
minute_appliances_status['Timestamp'] = (
            pd.DatetimeIndex(minute_appliances_status["Timestamp"]).astype(np.int64) / 1e9).astype(int)


In [264]:
minute_appliances_status["DayOfYear"] = (
            (minute_appliances_status['Timestamp'] - minute_appliances_status['Timestamp'].min()) / (3600 * 24)).astype(
    "int")

In [265]:
minute_appliances_status

,Timestamp,Load_StatusApplianceCooktop,Load_StatusApplianceDishwasher,Load_StatusApplianceOven,Load_StatusApplianceRangeHood,Load_StatusBA1Lights,Load_StatusBA2Lights,Load_StatusBR2Lights,Load_StatusBR3Lights,Load_StatusBR4Lights,...,Load_StatusSensHeatPrntADOWN,Load_StatusSensHeatPrntAUP,Load_StatusSensHeatPrntBDOWN,Load_StatusSensHeatPrntBUP,Load_StatusClothesWasher,Load_StatusDryerPowerTotal,KitchenLights,LivingRoomLights,MasterBedroomLights,DayOfYear
0,1372636874,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
1,1372636934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
2,1372636994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
3,1372637054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
4,1372637114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518787,1404170962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0,0,0.0,0.0,0.0,364
518788,1404171022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0,0,0.0,0.0,0.0,364
518789,1404171082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0,0,0.0,0.0,0.0,364
518790,1404172762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,364


In [266]:
appliance_job_list = []

for appliance in appliances_statusOnOff:
    # name = appliance.split("Load_Status")[-1]
    appliance_switch = minute_appliances_status[(minute_appliances_status[appliance] != minute_appliances_status[appliance].shift(1))][["Timestamp", "DayOfYear", appliance]][1:]

    df_indices = appliance_switch.index.tolist()
    for index, df_index in enumerate(df_indices):
        if appliance_switch.loc[df_index, appliance]:
            try:
                appliance_job_list.append({"start": appliance_switch.loc[df_index, "Timestamp"],
                                           "end": appliance_switch.loc[df_indices[index + 1], "Timestamp"],
                                           "appliance": appliance,
                                           "day": appliance_switch.loc[df_indices[index], "DayOfYear"]
                                           })
            except:
                continue

appliance_job_list

[{'start': 1372701631,
  'end': 1372702651,
  'appliance': 'Load_StatusApplianceCooktop',
  'day': 0},
 {'start': 1372788028,
  'end': 1372789048,
  'appliance': 'Load_StatusApplianceCooktop',
  'day': 1},
 {'start': 1372874427,
  'end': 1372875447,
  'appliance': 'Load_StatusApplianceCooktop',
  'day': 2},
 {'start': 1372960826,
  'end': 1372961846,
  'appliance': 'Load_StatusApplianceCooktop',
  'day': 3},
 {'start': 1373047228,
  'end': 1373048248,
  'appliance': 'Load_StatusApplianceCooktop',
  'day': 4},
 {'start': 1373098407,
  'end': 1373099367,
  'appliance': 'Load_StatusApplianceCooktop',
  'day': 5},
 {'start': 1373112027,
  'end': 1373113047,
  'appliance': 'Load_StatusApplianceCooktop',
  'day': 5},
 {'start': 1373137106,
  'end': 1373138126,
  'appliance': 'Load_StatusApplianceCooktop',
  'day': 5},
 {'start': 1373184807,
  'end': 1373185827,
  'appliance': 'Load_StatusApplianceCooktop',
  'day': 6},
 {'start': 1373198427,
  'end': 1373199447,
  'appliance': 'Load_StatusAp

In [267]:
df = pd.DataFrame(appliance_job_list).sort_values(by=['day', 'start']).reset_index(drop=True)
df

,start,end,appliance,day
0,1372649357,1372649537,Load_StatusMBALights,0
1,1372651097,1372651277,Load_StatusMBALights,0
2,1372658297,1372659197,Load_StatusMBALights,0
3,1372658297,1372660097,Load_StatusSensHeatPrntBDOWN,0
4,1372658297,1372666997,KitchenLights,0
...,...,...,...,...
28369,1404165562,1404172762,Load_StatusSensHeatPrntBUP,364
28370,1404165562,1404169162,MasterBedroomLights,364
28371,1404165862,1404167062,Load_StatusMBALights,364
28372,1404167362,1404167962,Load_StatusMBALights,364


In [268]:
df.to_csv(path_or_buf=Db.get_project_path("Project/04TPMAlgorithm/TPM/Data/houseDataWDryerAndWasher.csv"), header=False,
          index=False)

In [269]:
df1 = pd.read_csv(Db.get_project_path("Project/04TPMAlgorithm/TPM/Data/houseData.csv"), header=None)
df2 = pd.read_csv(Db.get_project_path("Project/04TPMAlgorithm/TPM/Data/houseDataWDryerAndWasher.csv"), header=None)

In [270]:
df1

,0,1,2,3
0,1372649357,1372649537,M B A Lights,0
1,1372651097,1372651277,M B A Lights,0
2,1372658297,1372659197,M B A Lights,0
3,1372658297,1372666997,Kitchen Lights,0
4,1372658597,1372659017,Plug Load Blender,0
...,...,...,...,...
18641,1404165562,1404172762,Plug Load Fan,364
18642,1404165562,1404166462,Plug Load Heating Pad,364
18643,1404165562,1404169162,Master Bedroom Lights,364
18644,1404165862,1404167062,M B A Lights,364


In [271]:
df2

,0,1,2,3
0,1372649357,1372649537,Load_StatusMBALights,0
1,1372651097,1372651277,Load_StatusMBALights,0
2,1372658297,1372659197,Load_StatusMBALights,0
3,1372658297,1372660097,Load_StatusSensHeatPrntBDOWN,0
4,1372658297,1372666997,KitchenLights,0
...,...,...,...,...
28369,1404165562,1404172762,Load_StatusSensHeatPrntBUP,364
28370,1404165562,1404169162,MasterBedroomLights,364
28371,1404165862,1404167062,Load_StatusMBALights,364
28372,1404167362,1404167962,Load_StatusMBALights,364


In [272]:
df_diff = pd.concat([df1, df2]).drop_duplicates(keep=False, ignore_index=True)

In [273]:
df_diff

,0,1,2,3
0,1372649357,1372649537,M B A Lights,0
1,1372651097,1372651277,M B A Lights,0
2,1372658297,1372659197,M B A Lights,0
3,1372658297,1372666997,Kitchen Lights,0
4,1372658597,1372659017,Plug Load Blender,0
...,...,...,...,...
47015,1404165562,1404172762,Load_StatusSensHeatPrntBUP,364
47016,1404165562,1404169162,MasterBedroomLights,364
47017,1404165862,1404167062,Load_StatusMBALights,364
47018,1404167362,1404167962,Load_StatusMBALights,364
